In [ ]:
###installing necessary libraries

In [ ]:
# import sys
# sys.path.append("../input/tez-lib/")

!pip install tez

In [ ]:
!pip install gdown

In [ ]:
!pip install git+https://github.com/rwightman/pytorch-image-models

In [ ]:
##preparing data

In [ ]:
#downloading Train data
!gdown https://drive.google.com/uc?id=1EtjM_HGKaWcHnxMvn33Ud0rKICt9dSKS

In [ ]:
#downloading Val data
!gdown https://drive.google.com/uc?id=1pnpG87i1uTvbMMxpsmnDuBEkQYR7n4Nb

In [ ]:
!unzip "./Train.zip"

In [ ]:
!unzip "./Val Blind.zip"

In [ ]:
###loading necessary libraries

In [ ]:
import  tez
import albumentations
import pandas as pd
import cv2
import numpy as np
import timm

import torch.nn as nn
from sklearn import metrics
import torch
from tez.callbacks import EarlyStopping
from tqdm import tqdm


In [ ]:
class args:
    batch_size = 8
    #image_size = 384
    image_size=384
    epochs =20
    fold=0

In [ ]:
##dataset class

In [ ]:
class CovidDataset:
    def __init__(self,image_paths,targets,augmentations):
        self.image_paths = image_paths
        self.targets = targets
        self.augmentations = augmentations
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self,item):
        image = cv2.imread(self.image_paths[item])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.augmentations is not  None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]
        
        image = np.transpose(image,(2,0,1)).astype(np.float32)
        
        targets = self.targets[item]
        
        return {
            "image":torch.tensor(image,dtype=torch.float),
            "targets":torch.tensor(targets,dtype=torch.float)
        }
    

In [ ]:
###designing model

In [ ]:
class CovidModel(tez.Model):
    def __init__(self):
        super().__init__()
        self.model = timm.create_model("swin_large_patch4_window12_384_in22k",pretrained=True,in_chans=3)
        self.model.head = nn.Linear(self.model.head.in_features,128)
        
        self.dropout = nn.Dropout(0.1)
        self.dense1 = nn.Linear(128,64)
        self.dense2 = nn.Linear(64, 1)
        self.step_scheduler_after = "epoch"
    
    def monitor_metrics(self,outputs,targets):
        outputs = outputs.cpu().detach().numpy()
        targets = targets.cpu().detach().numpy()
        rmse = metrics.mean_squared_error(targets,outputs,squared=False)
        return {"rmse":rmse}
    
    def fetch_scheduler(self):
        sch = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            self.optimizer,T_0 =10,T_mult=1, eta_min=1e-6,last_epoch=-1
               )
        return  sch 
    
    def fetch_optimizer(self):
        opt = torch.optim.Adam(self.parameters(),lr=1e-4)
        return opt
    
    def forward(self,image,targets=None):
        x = self.model(image)
        x = self.dropout(x)
        x = self.dense1(x)
        x = self.dense2(x)
        
        
        if targets is not None:
            loss = nn.MSELoss()(x,targets.view(-1,1))
            metrics = self.monitor_metrics(x,targets)
            return x,loss,metrics
        
        return x,0,{}

In [ ]:
train_aug = albumentations.Compose([
    albumentations.Resize(args.image_size,args.image_size,p=1),
    albumentations.HueSaturationValue(
        hue_shift_limit=0.2,
        sat_shift_limit =0.2,
        val_shift_limit=0.2,
        p=0.5
    ),
    albumentations.RandomBrightnessContrast(
        brightness_limit=(-0.1,0.1),
        contrast_limit =(-0.1,0.1),p=0.5
    ),

    albumentations.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
        max_pixel_value=255.0,
        p=1.0
    ),
],p=1.0)

valid_aug  = albumentations.Compose([
    
    albumentations.Resize(args.image_size,args.image_size,p=1),
    albumentations.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
        max_pixel_value=255.0,
        p=1.0,
    
    ),
],p=1.0)

In [ ]:
df = pd.read_csv("../input/covid-percentage/train_10folds.csv")
df_train = df[df.kfold != args.fold].reset_index(drop=True)
df_valid = df[df.kfold == args.fold].reset_index(drop=True)

In [ ]:
train_img_paths =[f"./Train/{x}" for x in df_train["name"].values]

In [ ]:
valid_img_paths =[f"./Train/{x}" for x in df_valid["name"].values]

In [ ]:
train_dataset = CovidDataset(
    image_paths = train_img_paths,
    targets= df_train.per.values,
    augmentations=train_aug
)
valid_dataset = CovidDataset(
    image_paths = valid_img_paths,
    targets = df_valid.per.values,
    augmentations=valid_aug 
)

In [ ]:
model= CovidModel()
es = EarlyStopping(
    monitor='valid_rmse',
    model_path = f"model_f{args.fold}.bin",
    patience =5,
    mode="min",
    save_weights_only=True,
)

In [ ]:
##starting the training process

In [ ]:
model.fit(
train_dataset,
valid_dataset=valid_dataset,
train_bs =args.batch_size,
valid_bs= args.batch_size,
device="cuda",
epochs=20,
callbacks=[es],
fp16=True)

In [ ]:
##evaluating best model on val data and preparing test file

In [ ]:
df_test= pd.read_csv("../input/first-model-percentage/val_file.csv")

In [ ]:
df_test.head()

In [ ]:

model.load("model_f0.bin", device="cuda", weights_only=True)


In [ ]:
test_img_paths =[f"./Val/{x}" for x in df_test["name"].values]

In [ ]:
len(test_img_paths)

In [ ]:
test_img_paths[0]

In [ ]:
test_dataset = CovidDataset(
        image_paths=test_img_paths,
        targets=np.ones(len(test_img_paths)),
        augmentations=valid_aug,
    )

In [ ]:
test_predictions = model.predict(test_dataset, batch_size=args.batch_size, n_jobs=-1)

In [ ]:
final_test_predictions = []
for preds in tqdm(test_predictions):
    final_test_predictions.extend(preds.ravel().tolist())

In [ ]:
l = [ 0 if x<=0 else x  for x in final_test_predictions]

In [ ]:

df_test["per"] =l
df_test = df_test[["name", "per"]]
df_test.to_csv("predictions.csv", index=False)